In [1]:
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta

# Alpaca API keys (replace with your keys)
API_KEY = 'PKA8CAYXHIJZVP6N16Y3'
API_SECRET = 'UfbITIZ9N3hPnoppHda2HjfFBkPom'
BASE_URL = 'https://paper-api.alpaca.markets'  # Paper trading URL

# Initialize Alpaca API
api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

# Define the stock symbol to trade (e.g., Apple)
symbol = 'AAPL'

# Fetch historical data (past 100 days)
def get_data(symbol, days=100):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    barset = api.get_barset(symbol, 'day', start=start_date, end=end_date)
    df = barset[symbol]
    df = pd.DataFrame({
        'timestamp': [bar.t.timestamp() for bar in df],
        'open': [bar.o for bar in df],
        'high': [bar.h for bar in df],
        'low': [bar.l for bar in df],
        'close': [bar.c for bar in df],
        'volume': [bar.v for bar in df],
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    return df

# Feature engineering: Creating simple features like moving averages
def feature_engineering(df):
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['RSI'] = 100 - (100 / (1 + df['close'].pct_change().rolling(window=14).mean() / df['close'].pct_change().rolling(window=14).std()))
    df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)  # 1 for buy, 0 for hold/sell
    df = df.dropna()
    return df

# Train a machine learning model
def train_model(df):
    X = df[['SMA_20', 'SMA_50', 'RSI']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'Model Accuracy: {accuracy_score(y_test, y_pred)}')
    return model

# Execute a trade (buy/sell)
def execute_trade(model, df, symbol):
    # Get the latest data point
    last_data = df.iloc[-1]
    X_latest = last_data[['SMA_20', 'SMA_50', 'RSI']].values.reshape(1, -1)
    
    # Predict the action
    action = model.predict(X_latest)[0]
    
    # Get the current account balance
    account = api.get_account()
    cash = float(account.cash)
    
    if action == 1 and cash > 10:  # Buy if predicted 1 (buy) and have enough cash
        print(f"Buying {symbol}...")
        api.submit_order(
            symbol=symbol,
            qty=1,  # Number of shares
            side='buy',
            type='market',
            time_in_force='gtc'  # Good till cancelled
        )
    elif action == 0:  # Sell if predicted 0 (hold/sell)
        print(f"Selling {symbol}...")
        positions = api.list_positions()
        for position in positions:
            if position.symbol == symbol:
                qty = int(position.qty)
                if qty > 0:
                    api.submit_order(
                        symbol=symbol,
                        qty=qty,
                        side='sell',
                        type='market',
                        time_in_force='gtc'
                    )

# Main function
def main():
    print(f"Fetching data for {symbol}...")
    data = get_data(symbol)
    print("Performing feature engineering...")
    data = feature_engineering(data)
    print("Training model...")
    model = train_model(data)
    
    # Execute trade based on the model's prediction
    execute_trade(model, data, symbol)

# Run the bot
if __name__ == '__main__':
    main()


Fetching data for AAPL...


AttributeError: 'REST' object has no attribute 'get_barset'

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
import numpy as np
from sklearn.metrics import accuracy_score

# Define the stock symbol to trade (e.g., Apple)
symbol = 'AAPL'

# Fetch historical data using Yahoo Finance (past 100 days)
def get_data(symbol, days=100):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    # Download historical data
    data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
    
    # Reset the index to get 'Date' as a column
    data.reset_index(inplace=True)
    
    # Rename columns for consistency with your original code
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    
    return data

# Feature engineering: Creating simple features like moving averages
def feature_engineering(df):
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['RSI'] = 100 - (100 / (1 + df['close'].pct_change().rolling(window=14).mean() / df['close'].pct_change().rolling(window=14).std()))
    df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)  # 1 for buy, 0 for hold/sell
    df = df.dropna()
    return df

# Train a machine learning model
def train_model(df):
    X = df[['SMA_20', 'SMA_50', 'RSI']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'Model Accuracy: {accuracy_score(y_test, y_pred)}')
    return model

# Execute a trade (buy/sell)
def execute_trade(model, df, symbol):
    # Get the latest data point
    last_data = df.iloc[-1]
    X_latest = last_data[['SMA_20', 'SMA_50', 'RSI']].values.reshape(1, -1)
    
    # Predict the action
    action = model.predict(X_latest)[0]
    
    # Simulating the trade process (as Alpaca API is unavailable)
    if action == 1:  # Buy if predicted 1 (buy)
        print(f"Buying {symbol}...")
    elif action == 0:  # Sell if predicted 0 (hold/sell)
        print(f"Selling {symbol}...")

# Main function
def main():
    print(f"Fetching data for {symbol}...")
    data = get_data(symbol)
    print("Performing feature engineering...")
    data = feature_engineering(data)
    print("Training model...")
    model = train_model(data)
    
    # Save the model using joblib
    joblib.dump(model, 'stock_trading_model.pkl')
    print("Model saved successfully!")
    
    # Execute trade based on the model's prediction
    execute_trade(model, data, symbol)

# Run the bot
if __name__ == '__main__':
    main()


NameError: name 'model' is not defined

In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the stock symbol to trade (e.g., Apple)
symbol = 'AAPL'

# Fetch historical data using Yahoo Finance (past 100 days)
def get_data(symbol, days=100):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    # Download historical data
    data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
    
    # Reset the index to get 'Date' as a column
    data.reset_index(inplace=True)
    
    # Rename columns for consistency with your original code
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    
    return data

# Feature engineering: Creating simple features like moving averages
def feature_engineering(df):
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['RSI'] = 100 - (100 / (1 + df['close'].pct_change().rolling(window=14).mean() / df['close'].pct_change().rolling(window=14).std()))
    df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)  # 1 for buy, 0 for hold/sell
    df = df.dropna()
    return df

# Train a deep learning model
def train_model(df):
    # Feature and target variables
    X = df[['SMA_20', 'SMA_50', 'RSI']]
    y = df['target']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Build a simple neural network model
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification (buy/sell)
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    
    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(f'Model Accuracy: {accuracy_score(y_test, y_pred)}')
    
    return model

# Execute a trade (buy/sell)
def execute_trade(model, data, symbol):
    # Prepare the latest data for prediction (ensure proper shape and dtype)
    X_latest = data[-1][['SMA_20', 'SMA_50', 'RSI']].values  # Adjust to your data format
    X_latest = np.array(X_latest, dtype=np.float32)

    # Ensure it's in the correct shape for prediction (1 sample, n features)
    if X_latest.ndim == 1:
        X_latest = X_latest.reshape(1, -1)

    print(f"X_latest shape: {X_latest.shape}")

    # Predict the action: 1 = buy, 0 = hold/sell
    prediction = model.predict(X_latest)
    action = (prediction > 0.5).astype(int)[0][0]

    print(f"Predicted action: {action}")

    # Execute trade based on the action
    if action == 1:
        print(f"Executing Buy order for {symbol}")
        # Execute Buy logic here
    else:
        print(f"Executing Sell/Hold order for {symbol}")
# Main function
def main():
    print(f"Fetching data for {symbol}...")
    data = get_data(symbol)
    print("Performing feature engineering...")
    data = feature_engineering(data)
    print("Training model...")
    model = train_model(data)
    
    # Execute trade based on the model's prediction
    execute_trade(model, data, symbol)
    model.save('stock_model.h5')

# Run the bot
if __name__ == '__main__':
    main()


Fetching data for AAPL...


[*********************100%***********************]  1 of 1 completed
c:\Users\Shreyansh Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Performing feature engineering...
Training model...
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7647 - loss: 9.2100 - val_accuracy: 0.8000 - val_loss: 6.8490
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7059 - loss: 10.5768 - val_accuracy: 0.8000 - val_loss: 5.6088
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.7647 - loss: 4.7220 - val_accuracy: 0.8000 - val_loss: 4.4114
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.7059 - loss: 5.1251 - val_accuracy: 0.8000 - val_loss: 3.2541
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5294 - loss: 6.4977 - val_accuracy: 0.8000 - val_loss: 2.1345
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5882 - loss: 6.5738 - val_accuracy: 0.8000 - val_loss: 1.1056
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.5294 - loss: 4.5638 - val_accuracy: 0.2000 - val_loss: 1.0330
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.6471 - los

KeyError: -1